## COVID-19 Ingestion pipeline

### Step 1: get COVID-19 data (DO NOT RUN)

In [ ]:
!mkdir covid-19-csv

In [ ]:
!cd covid-19-csv

In [ ]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

In [ ]:
gsutil cp COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/*.csv gs://covid-19-csv

### Step 2: load files into BQ (Start from here)

In [5]:
dataset_id = "covid_19_staging"

In [6]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'cs327e-sp2020:covid_19_staging' successfully created.


In [14]:
!bq --location=US load --skip_leading_rows=1 --allow_jagged_rows=true --source_format=CSV {dataset_id}.Daily_Cases \
gs://covid-19-csv/*.csv \
state:STRING,country:STRING,last_update:STRING,confirmed:INTEGER,deaths:INTEGER,recovered:INTEGER,latitude:NUMERIC,longitude:NUMERIC

Waiting on bqjob_r644b9b6b0bff9910_00000170c0b728ff_1 ... (5s) Current status: DONE   


In [16]:
%%bigquery
select count(*) as total_cases from covid_19_staging.Daily_Cases

,total_cases
0,4247


In [23]:
%%bigquery
select min(last_update) as oldest_timestamp, max(last_update) as newest_timestamp
from covid_19_staging.Daily_Cases

,oldest_timestamp,newest_timestamp
0,1/22/2020 17:00,2020-03-08T23:23:03


In [24]:
%%bigquery
select last_update as timestamp, count(*) as number_of_updates
from covid_19_staging.Daily_Cases
group by last_update
order by last_update desc
limit 10

,timestamp,number_of_updates
0,2020-03-08T23:23:03,1
1,2020-03-08T22:03:11,1
2,2020-03-08T22:03:10,1
3,2020-03-08T21:53:04,2
4,2020-03-08T21:53:03,1
5,2020-03-08T21:43:03,6
6,2020-03-08T21:33:02,2
7,2020-03-08T21:23:03,4
8,2020-03-08T21:13:14,2
9,2020-03-08T21:13:12,1
